In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow as tf
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import SGD , Adam , Adadelta
from keras.models import Sequential
from keras import backend as K
from keras.layers import Dropout , Dense , Activation , Flatten , Conv2D , MaxPooling2D , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator , load_img, img_to_array
from keras.callbacks import Callback

In [ ]:
train = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)'
test = "../input/new-plant-diseases-dataset/test"
img_width = 100
img_height = 100

train_samples = 87000
validation_sample = 33

epochs = 30
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height,3)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range =20,
    width_shift_range = 0.10,
    height_shift_range = 0.10,
    rescale = 1./255.,
    zoom_range = 0.1,
    horizontal_flip = True,
)

train_gen = datagen.flow_from_directory(
    train,
    target_size = (100,100),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical'
)

validation_gen = datagen.flow_from_directory(
    test,
    target_size = (100,100),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical'
)

In [ ]:
train_gen.class_indices  


In [ ]:
model= Sequential()

model.add(Conv2D (32,(3,3),input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D (64,(3,3),input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D (64,(3,3),input_shape= (100,100,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
opt = Adam(lr=0.001)

model.compile(loss = 'binary_crossentropy',optimizer = opt,metrics=['accuracy'])

In [ ]:
model.summary() 


In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=300,
                              validation_data = validation_gen,
                              validation_steps = validation_sample // batch_size,
                              epochs = 1)